<a href="https://colab.research.google.com/github/samaneh-m/TU-deep-Learning/blob/main/linear_layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Fully Connected Layers

In deep learning, a **fully connected layer** (or *linear layer*) is a fundamental building block of neural networks. It connects every neuron in the previous layer to every neuron in the current layer. This allows the network to learn complex relationships between features.

### Input and Output Shapes

Let the input tensor be denoted as:

$X \in \mathbb{R}^{N \times D_{\text{in}}}$

where:

- $N$: batch size
- $D_{\text{in}}$: number of input features

The fully connected layer applies a linear transformation to the input tensor, using a weight matrix $W$ and a bias vector $b$ with the following shapes:

$W \in \mathbb{R}^{D_{\text{in}} \times D_{\text{out}}}$

$b \in \mathbb{R}^{D_{\text{out}}}$

where:

- $D_{\text{out}}$: number of output features

The output tensor $Y$ has the shape:

$Y \in \mathbb{R}^{N \times D_{\text{out}}}$

### Linear Transformation

Each output value $Y[n, d_{\text{out}}]$ is computed as a weighted sum of the input features, plus a bias term:
$$
Y[n, d_{\text{out}}] = \left( \sum_{d_{\text{in}}=0}^{D_{\text{in}}-1} X[n, d_{\text{in}}] \cdot W[d_{\text{in}}, d_{\text{out}}] \right) + b[d_{\text{out}}]
$$

## Imports

In [ ]:
import torch
from torch import nn
from helper import *

## 🛠 Implement the Forward Pass (2 Loops)

🔹***Task:*** Implement the forward pass of a fully connected layer by using **two nested loops**.

🔍 **Note:** You are only allowed to use these PyTorch functions for your code. This is all you need:
- `torch.zeros`, `torch.sum`, `tensor.shape`
output[i,j]=
k
∑
​
 input[i,k]⋅weight[j,k]+bias[j]

Look into the [documentation](https://pytorch.org/docs/stable/torch.html) for a detailed function explanation. On the website, there is a searchbar at the top left.

In [ ]:
class MyLinearFunction_2_Loops(torch.autograd.Function):

    @staticmethod
    def forward(ctx, input, weight, bias):
        '''
        Arguments:
            input torch.Tensor -- input tensor of shape (N, D_in)
            weight torch.Tensor -- weight tensor of shape (D_out, D_in)
            bias torch.Tensor -- bias tensor of shape (D_out)

        Returns:
            output torch.Tensor -- output tensor of shape (N, D_out)
        '''
        ################################################
        N, D_in = input.shape
        D_out = weight.shape[0]
        output = torch.zeros(N, D_out)  # initialize the output

        for i in range(N):  # loop over each sample in the batch
            for d_out in range(D_out):  # loop over each output feature
                sum = 0
                for d_in in range(D_in):  # loop over input features
                    sum += input[i, d_in] * weight[d_out, d_in]
                output[i, d_out] = sum + bias[d_out]  # add the bias

        ################################################

        # Save tensors and parameters for backward pass
        ctx.save_for_backward(input, weight, bias)
        return output

    @staticmethod
    def backward(ctx, grad_output):
        return None

class MyLinear_2_Loops(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        """
        Initializes the custom Linear layer.
        """
        self.weight = nn.Parameter(torch.randn(out_features, in_features))
        self.bias = nn.Parameter(torch.ones(out_features))

    def forward(self, x):
        return MyLinearFunction_2_Loops.apply(x, self.weight, self.bias)

# test your implementation
test_linear_forward(MyLinear_2_Loops)

Testing with in_features=4, out_features=8
Testing with in_features=4, out_features=16
Testing with in_features=8, out_features=8
Testing with in_features=8, out_features=16
Nice! Forward pass is correct. Move to the next task.


## 🛠 Implement the Forward Pass (1 Loop)

🔹***Task:*** Implement the forward pass of a fully connected layer by using **one loop**. Loop over the batch size.

🔍 **Note:** You are only allowed to use these PyTorch functions for your code. This is all you need:
- `torch.zeros`, `torch.sum`, `tensor.shape`

Look into the [documentation](https://pytorch.org/docs/stable/torch.html) for a detailed function explanation. On the website, there is a searchbar at the top left.

In [ ]:
class MyLinearFunction_1_Loop(torch.autograd.Function):

    @staticmethod
    def forward(ctx, input, weight, bias):
        '''
        Arguments:
            input torch.Tensor -- input tensor of shape (N, D_in)
            weight torch.Tensor -- weight tensor of shape (D_out, D_in)
            bias torch.Tensor -- bias tensor of shape (D_out)

        Returns:
            output torch.Tensor -- output tensor of shape (N, D_out)
        '''
        ################################################
        N, D_in = input.shape
        D_out = weight.shape[0]
        output = torch.zeros(N, D_out)  # initialize the output
        for n in range(N):
            output[n] = torch.sum(input[n].unsqueeze(1) * weight.T, dim=0) + bias
        ################################################

        # Save tensors and parameters for backward pass
        ctx.save_for_backward(input, weight, bias)
        return output

    @staticmethod
    def backward(ctx, grad_output):
        return None

class MyLinear_1_Loop(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        """
        Initializes the custom Linear layer.
        """
        self.weight = nn.Parameter(torch.randn(out_features, in_features))
        self.bias = nn.Parameter(torch.ones(out_features))

    def forward(self, x):
        return MyLinearFunction_1_Loop.apply(x, self.weight, self.bias)

# test your implementation
test_linear_forward(MyLinear_1_Loop)

Testing with in_features=4, out_features=8
Testing with in_features=4, out_features=16
Testing with in_features=8, out_features=8
Testing with in_features=8, out_features=16
Nice! Forward pass is correct. Move to the next task.


## 🛠 Implement the Forward Pass (No Loops)

🔹***Task:*** Implement the forward pass of a fully connected layer by using **no loops**.

🔍 **Note:** You are only allowed to use these PyTorch functions for your code. This is all you need:
- `torch.matmul` or `@`, `tensor.T`

Look into the [documentation](https://pytorch.org/docs/stable/torch.html) for a detailed function explanation. On the website, there is a searchbar at the top left.

In [ ]:
class MyLinearFunction_No_Loops(torch.autograd.Function):

    @staticmethod
    def forward(ctx, input, weight, bias):
        '''
        Arguments:
            input torch.Tensor -- input tensor of shape (N, D_in)
            weight torch.Tensor -- weight tensor of shape (D_out, D_in)
            bias torch.Tensor -- bias tensor of shape (D_out)

        Returns:
            output torch.Tensor -- output tensor of shape (N, D_out)
        '''
        ################################################
        N, D_in = input.shape
        D_out = weight.shape[0]
        output = torch.zeros(N, D_out)
        # Initialize the output tensor with the correct shape

        # Matrix multiplication: input @ weight.T → (N, D_out)
        output = input @ weight.T + bias  # Broadcasting handles the bias add
        ################################################

        # Save tensors and parameters for backward pass
        ctx.save_for_backward(input, weight, bias)
        return output

    @staticmethod
    def backward(ctx, grad_output):
        return None

class MyLinear_No_Loops(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        """
        Initializes the custom Linear layer.
        """
        self.weight = nn.Parameter(torch.randn(out_features, in_features))
        self.bias = nn.Parameter(torch.ones(out_features))

    def forward(self, x):
        return MyLinearFunction_No_Loops.apply(x, self.weight, self.bias)

# test your implementation
test_linear_forward(MyLinear_No_Loops)

Testing with in_features=4, out_features=8
Testing with in_features=4, out_features=16
Testing with in_features=8, out_features=8
Testing with in_features=8, out_features=16
Nice! Forward pass is correct. Move to the next task.
